# Crime Patterns in Chicago
*Examining the Relationship Between Daytime and Nighttime Crime Rates*

## Setup

### Imports

In [ ]:
!pip install astral

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
# from google.colab import drive
from pyproj import Transformer
from sklearn.preprocessing import LabelEncoder
from astral import LocationInfo
from astral.sun import sun
from scipy.stats import pearsonr
from IPython.display import display
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.cluster import KMeans
import warnings
warnings.filterwarnings("ignore")

### Load Data

In [ ]:
drive.mount('/content/drive')

# NOTE: To get this working, right click the 'In Data We Trust' folder in
#   Google Drive, then add a shortcut. This will then work automatically
#   without having to change the directory.
proj_dir = '/content/drive/MyDrive/CS326 - In Data We Trust'

# This should print the files in the project folder.
!ls "$proj_dir"

In [ ]:
# Fix columns names
df = pd.read_csv("crimes_data_chicago.csv")
df.columns = df.columns.str.replace(r'\s+', ' ', regex=True)\
                      .str.strip().str.lower()\
                      .str.replace(' ', '_').str.replace('#', '')
df.columns

In [ ]:
df.head()

## Data Cleaning

**NOTE:** PROVIDE REASONING FOR THIS

In [ ]:
# Drop additional columns

columns_to_drop = ['case', 'x_coordinate', 'y_coordinate', 'location', 'iucr']
df = df.drop(columns_to_drop, axis=1)

Quick analysis of what percentage of rows contain NA location

In [ ]:
prev_num_rows = len(df.index)

# There are 93 rows where NaN values are in latitude and longitude
df = df.dropna(subset=['longitude', 'latitude'])

# We only have 1 row that has NaN value that is NOT latitude or longitude
# @ index 230265 for Ahmed (Hamood)
#   Drop unnecessary columns for modeling/correlation matrix right before preprocessing step
#   Save this as a separate dataframe!!!
df = df.dropna(subset=['ward'])

curr_num_rows = len(df.index)

print(curr_num_rows/prev_num_rows*100)

Grouping of less frequent categories under OTHER

In [ ]:
threshold = 0.01

# Display values for location_description, primary_description, secondary_description, fbi_cd
categorical_cols_to_aggr = [ "location_description", "primary_description",  "secondary_description", "fbi_cd"]
for col in categorical_cols_to_aggr:
    uniques = df[col].value_counts()
    # print(f"Unique values in {col}: {len(uniques)}")

    counts = df[col].value_counts(normalize=True)
    to_keep = counts[counts > threshold].index

    df.loc[:, col] = df[col].where(df[col].isin(to_keep), "OTHER")

    display(df[[col]].value_counts())


Creating day/time binary column

In [ ]:
df["date_of_occurrence"] = pd.to_datetime(df["date_of_occurrence"])
city = LocationInfo("Chicago", "USA", "America/Chicago", 41.8781, -87.6298)

def is_daytime(ts):
  # Check if the timestamp is NaT before localizing
  if pd.isna(ts):
    return 0 # Or handle missing timestamps as appropriate for your analysis

  # Localize the timestamp to the city's timezone, handling ambiguous times by setting them to NaT
  ts_localized = ts.tz_localize(city.timezone, ambiguous='NaT')

  # Check if localization resulted in NaT (due to ambiguity or original NaT)
  if pd.isna(ts_localized):
      return 0 # Or handle as appropriate

  # Get sunrise and sunset for the date of the localized timestamp
  s = sun(city.observer, date=ts_localized.date(), tzinfo=city.timezone)

  # Check if sunrise or sunset is NaT
  if pd.isna(s["sunrise"]) or pd.isna(s["sunset"]):
      return 0 # Or handle as appropriate

  return int(s["sunrise"] <= ts_localized <= s["sunset"])

# Apply the function to the date_of_occurrence column
df["is_daytime"] = df["date_of_occurrence"].apply(is_daytime)

In [ ]:
df.head(3)

In [ ]:
df.loc[:, 'arrest'] = df['arrest'].map({'Y': True, 'N': False})
df.loc[:, 'domestic'] = df['domestic'].map({'Y': True, 'N': False})

## Exploratory Data Analysis

In [ ]:
# @title Function for displaying correlation
def show_day_night_correlations(df, target_col, loc_col, daytime_col='is_daytime'):
    # 1. Ensure target is numeric (0 or 1) for the entire operation
    # We create a copy so we don't modify your original dataframe
    work_df = df.copy()
    work_df[target_col] = work_df[target_col].astype(int)

    # 2. TABLE: Calculate correlations split by Day/Night
    results = {}
    locations = work_df[loc_col].unique()

    for loc in locations:
        loc_binary = (work_df[loc_col] == loc).astype(int)

        # Day Stats
        mask_day = work_df[daytime_col] == True
        if mask_day.sum() > 0:
            r_day, p_day = pearsonr(loc_binary[mask_day], work_df.loc[mask_day, target_col])
        else:
            r_day, p_day = 0, 1.0

        # Night Stats
        mask_night = work_df[daytime_col] == False
        if mask_night.sum() > 0:
            r_night, p_night = pearsonr(loc_binary[mask_night], work_df.loc[mask_night, target_col])
        else:
            r_night, p_night = 0, 1.0

        results[loc] = {
            "Corr_Day": r_day, "P_Day": p_day,
            "Corr_Night": r_night, "P_Night": p_night,
            "Diff (Day-Night)": r_day - r_night
        }

    results_df = pd.DataFrame(results).T.sort_values("Diff (Day-Night)", ascending=False)

    def style_sig(val):
        return 'color: red' if val >= 0.01 else 'color: green'

    display(results_df.style.map(style_sig, subset=['P_Day', 'P_Night'])
            .format("{:.3f}")
            .background_gradient(subset=['Diff (Day-Night)'], cmap='coolwarm'))

    # 3. VISUALIZATION: Heatmap
    # Now using 'work_df' where target_col is guaranteed to be numeric
    pivot_df = work_df.pivot_table(
        index=loc_col,
        columns=daytime_col,
        values=target_col,
        aggfunc='mean'
    )

    pivot_df.columns = [f'Night ({target_col} rate)', f'Day ({target_col} rate)']

    plt.figure(figsize=(8, len(pivot_df) * 0.4 + 2))
    sns.heatmap(pivot_df, annot=True, cmap="Reds", fmt=".1%", cbar_kws={'label': 'Probability'})
    plt.title(f"Impact of {daytime_col} on {target_col} by Location")
    plt.ylabel("Location")
    plt.show()

In [ ]:
show_day_night_correlations(df, 'arrest', 'location_description')

## Modeling

### Data Preparation

One-hot and feature hash categorical columns,
EXCEPT `block` because it has over 28000 unique values and can be represented using `lat` and `lon`.

In [ ]:
real = df.copy()

In [ ]:
df = real.copy()
df = df.dropna()

# Time features 
df['month'] = df['date_of_occurrence'].dt.month
df['weekday'] = df['date_of_occurrence'].dt.weekday
df['is_weekend'] = df['weekday'].isin([5, 6]).astype(int)


df['crime_day_combo'] = df['primary_description'] + "_" + df['weekday'].astype(str)
df['loc_day_combo'] = df['location_description'] + "_" + df['weekday'].astype(str)

# GEO CLUSTER FEATURE 
coords = df[['latitude', 'longitude']].copy()
coords = coords.fillna(coords.median())  # safety

kmeans = KMeans(n_clusters=20, random_state=42, n_init=10)
df['geo_cluster'] = kmeans.fit_predict(coords)


# CATEGORICAL ENCODING 
categorical_cols = [
    'location_description',
    'beat',
    'ward',
    'fbi_cd',
    'crime_day_combo',
    'loc_day_combo',
    'geo_cluster',
]

ohe = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
encoded = ohe.fit_transform(df[categorical_cols])
encoded_df = pd.DataFrame(encoded, columns=ohe.get_feature_names_out(categorical_cols))

df_encoded = pd.concat([
    df.drop(columns=categorical_cols),
    encoded_df
], axis=1)

df_encoded = df_encoded.dropna(axis=0).reset_index(drop=True)

y = df_encoded['is_daytime']
X = df_encoded.drop(columns=['block', 'is_daytime', 'date_of_occurrence', 'primary_description', 'secondary_description'])

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


### Helper Functions

In [ ]:
def cross_validate_model(model):
    cr = cross_validate(
        model,
        X_train,
        y_train,
        scoring=['f1', 'roc_auc', 'accuracy'],
        n_jobs=-1,
        cv=5
    )

    print(f"Mean F1 Score: {cr['test_f1'].mean()}")
    print(f"Mean ROC AUC: {cr['test_roc_auc'].mean()}")
    print(f"Mean Accuracy: {cr['test_accuracy'].mean()}")

    return cr

In [ ]:
def grid_search(base_estimator, params):
    grid = GridSearchCV(
        estimator=base_estimator,
        param_grid=params,
        scoring={
            'f1': 'f1',
            'roc_auc': 'roc_auc',
            'accuracy': 'accuracy'
        },
        refit='f1',
        cv=3,
        n_jobs=-1,
        verbose=2
    )

    grid.fit(X_train, y_train)

    print("Best params this round:", grid.best_params_)

    best_idx = grid.best_index_
    mean_f1 = grid.cv_results_['mean_test_f1'][best_idx]
    mean_roc = grid.cv_results_['mean_test_roc_auc'][best_idx]
    mean_acc = grid.cv_results_['mean_test_accuracy'][best_idx]

    print(f"Best F1 (CV):       {mean_f1:.4f}")
    print(f"Best ROC AUC (CV):  {mean_roc:.4f}")
    print(f"Best Accuracy (CV): {mean_acc:.4f}")

    return grid

### KNN

In [ ]:
knn = cross_validate_model(KNeighborsClassifier(n_neighbors=1))

### Logistic Regression

In [ ]:
lr = cross_validate_model(LogisticRegression())

### Decision Tree

In [ ]:
dt = cross_validate_model(DecisionTreeClassifier())

### Random Forest

In [ ]:
rf_cv = cross_validate_model(RandomForestClassifier())

#### Grid Search

In [ ]:
rf = RandomForestClassifier(
    random_state=42,
    n_jobs=-1
)

In [ ]:
param_grid_rf_round_1 = {
    'n_estimators': [400, 300],
}

grid1 = grid_search(rf, param_grid_rf_round_1)
best_n_estimators = grid1.best_params_['n_estimators']


param_grid_rf_round_2 = {
    'n_estimators': [best_n_estimators],
    'max_depth': [None, 20],
    'max_features': ['sqrt', 0.5],
}

grid2 = grid_search(rf, param_grid_rf_round_2)
best_max_depth = grid2.best_params_['max_depth']
best_max_features = grid2.best_params_['max_features']


param_grid_rf_round_3 = {
    'n_estimators': [best_n_estimators],
    'max_depth': [best_max_depth],
    'max_features': [best_max_features],
    'min_samples_split': [2, 10],
    'min_samples_leaf': [1, 5],
}

grid3 = grid_search(rf, param_grid_rf_round_3)
best_min_split = grid3.best_params_['min_samples_split']
best_min_leaf = grid3.best_params_['min_samples_leaf']


param_grid_rf_round_4 = {
    'n_estimators': [best_n_estimators],
    'max_depth': [best_max_depth],
    'max_features': [best_max_features],
    'min_samples_split': [best_min_split],
    'min_samples_leaf': [best_min_leaf],
    'bootstrap': [True],
    'max_samples': [None, 0.7],
}


grid4 = grid_search(rf, param_grid_rf_round_4)
best_bootstrap = grid4.best_params_['bootstrap']
best_max_samples = grid4.best_params_.get('max_samples', None)


param_grid_rf_round_5 = {
    'n_estimators': [best_n_estimators],
    'max_depth': [best_max_depth],
    'max_features': [best_max_features],
    'min_samples_split': [best_min_split],
    'min_samples_leaf': [best_min_leaf],
    'bootstrap': [best_bootstrap],
    'max_samples': [best_max_samples],
    'class_weight': [None, 'balanced']
}

grid5 = grid_search(rf, param_grid_rf_round_5)
best_class_weight = grid5.best_params_['class_weight']


In [ ]:
param_grid_rf_final = {
                   
}

final_grid = grid_search(rf, param_grid_rf_final)
best_class_weight = grid5.best_params_

### Hist Gradient Boosting

In [ ]:
hgb = HistGradientBoostingClassifier(
    loss='log_loss',
    validation_fraction=0.1,
    n_iter_no_change=10,
    random_state=42
)

In [ ]:
hg_cv = cross_validate_model(hgb)

#### Grid search

In [ ]:
hgb = HistGradientBoostingClassifier(
    loss='log_loss',
    validation_fraction=0.1,
    n_iter_no_change=10,
    random_state=42
)

In [ ]:
param_grid_hgb_round_1 = {
    'learning_rate': [0.03, 0.05, 0.1, 0.2],
}
grid1 = grid_search(hgb, param_grid_hgb_round_1)
best_lr = grid1.best_params_['learning_rate']


param_grid_hgb_round_2 = {
    'learning_rate': [best_lr],
    'max_leaf_nodes': [31, 63, 127, 255],
    'max_depth': [None, 5, 10, 15],
}
grid2 = grid_search(hgb, param_grid_hgb_round_2)
best_leaf_nodes = grid2.best_params_['max_leaf_nodes']
best_depth = grid2.best_params_['max_depth']


param_grid_hgb_round_3 = {
    'learning_rate': [best_lr],
    'max_leaf_nodes': [best_leaf_nodes],
    'max_depth': [best_depth],
    'min_samples_leaf': [10, 20, 50, 100, 200],
}
grid3 = grid_search(hgb, param_grid_hgb_round_3)
best_min_leaf = grid3.best_params_['min_samples_leaf']


param_grid_hgb_round_4 = {
    'learning_rate': [best_lr],
    'max_leaf_nodes': [best_leaf_nodes],
    'max_depth': [best_depth],
    'min_samples_leaf': [best_min_leaf],
    'l2_regularization': [0.0, 0.5, 1.0, 2.0],
}
grid4 = grid_search(hgb, param_grid_hgb_round_4)
best_l2 = grid4.best_params_['l2_regularization']


param_grid_hgb_round_5 = {
    'learning_rate': [best_lr],
    'max_leaf_nodes': [best_leaf_nodes],
    'max_depth': [best_depth],
    'min_samples_leaf': [best_min_leaf],
    'l2_regularization': [best_l2],
    'class_weight': [None, 'balanced']
}
grid5 = grid_search(hgb, param_grid_hgb_round_5)
best_class_weight = grid5.best_params_['class_weight']

In [ ]:
param_grid_hgb_final = {
    'learning_rate': [0.02, 0.03, 0.04],        
    'max_leaf_nodes': [48, 63, 80],             
    'min_samples_leaf': [15, 20, 30],    
    'max_depth': [4, 5, 6],                                  
    'l2_regularization': [0.5],     
    'class_weight': [None],                     
}

final_grid = grid_search(hgb, param_grid_hgb_final)
best_class_weight = grid5.best_params_